In [213]:
from IPython.display import SVG
import drawsvg as draw
import math

class RegularPolygon:
    def __init__(self, vertices=3,canvas_size=100,vertex_size=2,style=1,canvas=None):
        self.canvas_size=canvas_size
        self.vertex_size=vertex_size
        self.slack=10
        if canvas:
            self.d = canvas
        else:
            self.d = draw.Drawing(self.canvas_size+self.slack, self.canvas_size+self.slack)
            self.d.set_pixel_scale(2)
        self.vertices=vertices
        self.U=[]
    def main(self):
        U=[]
        N=self.vertices
        r=(self.d.width-self.slack)/2
        for i in range(N):
            x = r*math.cos(i * (2*math.pi/N))
            y = r*math.sin(i * (2*math.pi/N))
            # offset it to avoid the canvas's edge
            x+=r+5
            y+=r+5
            U+=[x,y]
        self.d.append(draw.Lines(*U,close=True,fill='white',stroke='black'))
        self.U = U
    def draw_vertices1(self):
        U = self.U
        N = self.vertices
        for i in range(N):
            self.d.append(draw.Circle(U[2*i],U[2*i+1],self.vertex_size,fill='black', stroke_width=1, stroke='black'))            
    def diagonal1(self, p, q, color="black"):
        U=self.U
        D=list(map(lambda i: U[i], [2*p,2*p+1,2*q,2*q+1]))
        self.d.append(draw.Lines(*D,close=False,fill='white',stroke=color))
    def draw_vertices2(self):
        U = self.U
        N = self.vertices
        for i in range(N):
            self.d.append(draw.Circle(U[2*i],U[2*i+1],self.vertex_size,fill='white', stroke_width=0.4, stroke='black'))
            self.d.append(draw.Text(str(int(i)),4,U[2*i],U[2*i+1],fill='black',text_anchor='middle',dominant_baseline="middle"))
    def diagonal2(self, p, q, color="black"):
        U=self.U
        r=self.vertex_size
        dx = U[2*p]  -U[2*q]
        dy = U[2*p+1]-U[2*q+1]
        a=math.atan2(dy,dx)+math.pi
        x1=U[2*p]   + r*math.cos(a)
        y1=U[2*p+1] + r*math.sin(a)
        x2=U[2*q]   + r*math.cos(a+math.pi)
        y2=U[2*q+1] + r*math.sin(a+math.pi)
        D=[x1,y1,x2,y2]
        self.d.append(draw.Lines(*D,close=False,fill='white',stroke=color))
    def draw(self):
        return self.d.display_inline()


svgbuf=""
N=5
for i in range(N):
    p = RegularPolygon(vertices=N,vertex_size=3,canvas_size=50)
    p.main()
    p.draw_vertices2()
    u=i
    v=(u+2)%N
    p.diagonal2(u,v)
    svgbuf+=p.d.as_svg()

from IPython.display import display, HTML
no_wrap_div = '<div style="">{}</div>'
display(HTML(no_wrap_div.format(svgbuf)))

In [212]:
# DiagonalIter receives a list of labels for the vertices of an n-gon
# and it returns all its diagonals
class DiagonalIter:
    def __init__(self, L):
        self.L=L
    def __iter__(self):
        N=len(self.L)
        L=self.L
        for x in range(N):
            y1 = (x-2)%N
            y2 = (x+2)%N
            while True:
                if x<y2:
                    yield [L[x],L[y2]]
                y2 = (y2+1)%N
                if y2 == (y1+1)%N:
                    break

N=7
svgbuf=""
iter1 = DiagonalIter(list(range(N)))
seen=dict()
for d1 in iter1:
    x,y=d1

    # left split
    z,L=x,[]
    while z!=y:
        L+=[z]
        z=(z+1)%N
    L+=[y]

    # right split
    z,R=x,[]
    while z!=y:
        R+=[z]
        z=(z-1)%N
    R+=[y]

    S=[]
    for d2 in DiagonalIter(L):
        if set(d2).isdisjoint(d1):
            k = frozenset((frozenset(d1),frozenset(d2)))
            if k in seen: continue
            S+=[[d1,d2]]
            seen[k]=1

    for d2 in DiagonalIter(R):
        if set(d2).isdisjoint(d1):
            k = frozenset((frozenset(d1),frozenset(d2)))
            if k in seen: continue
            S+=[[d1,d2]]
            seen[k]=1

    for ds in S:
        p = RegularPolygon(vertices=N,vertex_size=3,canvas_size=50)
        p.main()
        p.draw_vertices2()
        p.diagonal2(*ds[0], color="blue")
        p.diagonal2(*ds[1], color="red")
        svgbuf+=p.d.as_svg()

from IPython.display import display, HTML
no_wrap_div = '<div style="">{}</div>'
raw_html = no_wrap_div.format(svgbuf)
display(HTML(raw_html))